### import libraries

#### import via ivy

In [1]:
import $ivy.`org.scalanlp:breeze_2.12:0.13.2`
import $ivy.`org.jfree:jfreechart:1.5.0`
import $ivy.`org.apache.spark:spark-core_2.11:2.4.3`
import $ivy.`org.apache.spark:spark-sql_2.11:2.4.3`
import $ivy.`org.apache.spark:spark-mllib_2.11:2.4.3`
import $ivy.`org.apache.spark:spark-graphx_2.11:2.4.3`
import $ivy.`org.apache.spark:spark-streaming_2.11:2.4.3`
import $ivy.`sh.almond::ammonite-spark:0.3.0`

#### Scala Library for Matrix Calculation

In [1]:
import breeze.linalg.csvread
import breeze.linalg.{DenseMatrix => DM}
import breeze.linalg.{DenseVector => DV}
import breeze.linalg

#### File IO

In [58]:
import java.io.File
import scala.io.Source
import java.io.{BufferedReader, InputStreamReader, FileInputStream}

import java.io.File

import scala.io.Source

import java.io.{BufferedReader, InputStreamReader, FileInputStream}

#### Spark Libraries

In [118]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkConf
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Column
import org.apache.spark.sql.catalyst.plans.{Inner, LeftOuter, Cross}

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.SparkSession

import org.apache.spark.SparkConf

import org.apache.spark.sql.functions._

import org.apache.spark.sql.Column

import org.apache.spark.sql.catalyst.plans.{Inner, LeftOuter, Cross}

#### Set level or warning statemens

In [28]:
import org.apache.log4j.Logger
import org.apache.log4j.Level
Logger.getLogger("org").setLevel(Level.ERROR)

import org.apache.spark.sql.functions._

import org.apache.spark.sql.catalyst.plans.{Inner, LeftOuter, Cross}

#### Specify folder & file paths 

In [2]:
val folder = new File("/Users/tomoya/Documents/dataset/champs-scalar-coupling")
val files = folder.listFiles
val trainFile = files.filter(x => x.getName.contains("train.csv")).head

folder: File = /Users/tomoya/Documents/dataset/champs-scalar-coupling
files: Array[File] = Array(
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/.DS_Store,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/dipole_moments.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/magnetic_shielding_tensors.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/mulliken_charges.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/potential_energy.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/sample_submission.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/scalar_coupling_contributions.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/structures.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/structures.zip,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/test.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/train.csv
)
trainFile: File = /Users/tomoya/Documents/dataset/champs

#### Reading file by java.io

In [59]:
// Function to read csv file (UTF-8)
def readHead(path:File, nline:Int):Seq[String] = {
    val reader = new BufferedReader(new InputStreamReader(new FileInputStream(path), "UTF-8"))
    try {
        Iterator.continually(reader.readLine()).take(nline).toList // Not toSeq
    }finally{
        reader.close()
    }
}

defined function readHead

In [44]:
val list = readHead(trainFile, 10000)

list: Seq[String] = List(
  "id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant",
  "0,dsgdb9nsd_000001,1,0,1JHC,84.8076",
  "1,dsgdb9nsd_000001,1,2,2JHH,-11.257",
  "2,dsgdb9nsd_000001,1,3,2JHH,-11.2548",
  "3,dsgdb9nsd_000001,1,4,2JHH,-11.2543",
  "4,dsgdb9nsd_000001,2,0,1JHC,84.8074",
  "5,dsgdb9nsd_000001,2,3,2JHH,-11.2541",
  "6,dsgdb9nsd_000001,2,4,2JHH,-11.2548",
  "7,dsgdb9nsd_000001,3,0,1JHC,84.8093",
  "8,dsgdb9nsd_000001,3,4,2JHH,-11.2543",
  "9,dsgdb9nsd_000001,4,0,1JHC,84.8095",
  "10,dsgdb9nsd_000002,1,0,1JHN,32.6889",
  "11,dsgdb9nsd_000002,1,2,2JHH,-11.1866",
  "12,dsgdb9nsd_000002,1,3,2JHH,-11.1757",
  "13,dsgdb9nsd_000002,2,0,1JHN,32.6891",
  "14,dsgdb9nsd_000002,2,3,2JHH,-11.1758",
  "15,dsgdb9nsd_000002,3,0,1JHN,32.6905",
  "16,dsgdb9nsd_000003,1,2,2JHH,-9.94641",
  "17,dsgdb9nsd_000005,2,0,1JHC,171.22",
  "18,dsgdb9nsd_000005,2,1,2JHN,5.18246",
  "19,dsgdb9nsd_000007,2,0,1JHC,83.5429",
  "20,dsgdb9nsd_000007,2,1,2JHC,-2.37831",
  "21,dsgdb9nsd

In [45]:
val cols = list(0).split(",")

cols: Array[String] = Array(
  "id",
  "molecule_name",
  "atom_index_0",
  "atom_index_1",
  "type",
  "scalar_coupling_constant"
)

#### Init Spark

In [46]:
val sc = new SparkConf
sc.setAppName("chemical").setMaster("local[*]")
val spark = SparkSession.builder().config(sc).getOrCreate()

19/07/14 15:59:04 WARN SparkSession$Builder: Using an existing SparkSession; some configuration may not take effect.


sc: SparkConf = org.apache.spark.SparkConf@312f4e35
res45_1: SparkConf = org.apache.spark.SparkConf@312f4e35
spark: SparkSession = org.apache.spark.sql.SparkSession@700161f

In [47]:
def toRecord(line:String) = {
    val strs = line.split(",")
    (strs(0).toInt, strs(1), strs(2).toInt, strs(3).toInt, strs(4), strs(5).toDouble)
}

defined function toRecord

In [48]:
val sampleDF = spark.createDataFrame((1 to 999).map(x => list(x)).map(x => toRecord(x))).toDF(cols:_*)

sampleDF: DataFrame = [id: int, molecule_name: string ... 4 more fields]

#### Drop unneeded column information

In [62]:
val sampleDF_2 = sampleDF.select(col("*") +: ((1 to 4).map(x => col("type").substr(x, 1).as("t1_" + x.toString)) ++
                            (1 to 3).map(x => col("type").substr(x, 2).as("t2_" + x.toString))):_*)

sampleDF_2: DataFrame = [id: int, molecule_name: string ... 11 more fields]

In [63]:
sampleDF_2.show()

+---+----------------+------------+------------+----+------------------------+----+----+----+----+----+----+----+
| id|   molecule_name|atom_index_0|atom_index_1|type|scalar_coupling_constant|t1_1|t1_2|t1_3|t1_4|t2_1|t2_2|t2_3|
+---+----------------+------------+------------+----+------------------------+----+----+----+----+----+----+----+
|  0|dsgdb9nsd_000001|           1|           0|1JHC|                 84.8076|   1|   J|   H|   C|  1J|  JH|  HC|
|  1|dsgdb9nsd_000001|           1|           2|2JHH|                 -11.257|   2|   J|   H|   H|  2J|  JH|  HH|
|  2|dsgdb9nsd_000001|           1|           3|2JHH|                -11.2548|   2|   J|   H|   H|  2J|  JH|  HH|
|  3|dsgdb9nsd_000001|           1|           4|2JHH|                -11.2543|   2|   J|   H|   H|  2J|  JH|  HH|
|  4|dsgdb9nsd_000001|           2|           0|1JHC|                 84.8074|   1|   J|   H|   C|  1J|  JH|  HC|
|  5|dsgdb9nsd_000001|           2|           3|2JHH|                -11.2541|   2|   J|

In [64]:
val summaries = sampleDF_2.summary()

summaries: DataFrame = [summary: string, id: string ... 12 more fields]

In [65]:
summaries.show()

+-------+-----------------+----------------+-----------------+------------------+----+------------------------+------------------+----+----+----+----+----+----+
|summary|               id|   molecule_name|     atom_index_0|      atom_index_1|type|scalar_coupling_constant|              t1_1|t1_2|t1_3|t1_4|t2_1|t2_2|t2_3|
+-------+-----------------+----------------+-----------------+------------------+----+------------------------+------------------+----+----+----+----+----+----+
|  count|              999|             999|              999|               999| 999|                     999|               999| 999| 999| 999| 999| 999| 999|
|   mean|            499.0|            null|6.367367367367367|3.2412412412412412|null|       20.20008205105104|2.1731731731731734|null|null|null|null|null|null|
| stddev|288.5307609250702|            null|2.370953186843761|3.1443018508277945|null|       41.64950520655079|0.7537754279841617|null|null|null|null|null|null|
|    min|                0|dsgdb9n

### Read whole file by spark

In [66]:
val df = spark.read.option("header", "true").csv(trainFile.getAbsolutePath)

df: DataFrame = [id: string, molecule_name: string ... 4 more fields]

In [92]:
var df2 = df.select(col("*") +: ((1 to 4).map(x => col("type").substr(x, 1).as("t1_" + x.toString)) ++
                            (1 to 3).map(x => col("type").substr(x, 2).as("t2_" + x.toString))):_*)

df2: DataFrame = [id: string, molecule_name: string ... 11 more fields]

In [93]:
val df2_summary = df2.summary()

df2_summary: DataFrame = [summary: string, id: string ... 12 more fields]

In [94]:
df2_summary.show()

+-------+------------------+----------------+------------------+------------------+-------+------------------------+------------------+-------+-------+-------+-------+-------+-------+
|summary|                id|   molecule_name|      atom_index_0|      atom_index_1|   type|scalar_coupling_constant|              t1_1|   t1_2|   t1_3|   t1_4|   t2_1|   t2_2|   t2_3|
+-------+------------------+----------------+------------------+------------------+-------+------------------------+------------------+-------+-------+-------+-------+-------+-------+
|  count|           4658147|         4658147|           4658147|           4658147|4658147|                 4658147|           4658147|4658147|4658147|4658147|4658147|4658147|4658147|
|   mean|         2329073.0|            null|13.356885688665471|5.8839663067739165|   null|       15.92164991825926| 2.325156333623649|   null|   null|   null|   null|   null|   null|
| stddev|1344691.3565249834|            null|3.2677124094357053| 4.9939430981278

information in t1_2, t1_3 can be dropped

#### Unique values in id, molecule name in df

In [95]:
df2.count

res94: Long = 4658147L

In [96]:
df2.select("id").dropDuplicates.count

res95: Long = 4658147L

In [97]:
df2.select("molecule_name").dropDuplicates.count

res96: Long = 85003L

In [98]:
df2 = df2.drop("t1_2", "t1_3", "t2_2", "t2_2", "t2_1", "t2_3", "type")

In [99]:
df2.show()

+---+----------------+------------+------------+------------------------+----+----+
| id|   molecule_name|atom_index_0|atom_index_1|scalar_coupling_constant|t1_1|t1_4|
+---+----------------+------------+------------+------------------------+----+----+
|  0|dsgdb9nsd_000001|           1|           0|                 84.8076|   1|   C|
|  1|dsgdb9nsd_000001|           1|           2|                 -11.257|   2|   H|
|  2|dsgdb9nsd_000001|           1|           3|                -11.2548|   2|   H|
|  3|dsgdb9nsd_000001|           1|           4|                -11.2543|   2|   H|
|  4|dsgdb9nsd_000001|           2|           0|                 84.8074|   1|   C|
|  5|dsgdb9nsd_000001|           2|           3|                -11.2541|   2|   H|
|  6|dsgdb9nsd_000001|           2|           4|                -11.2548|   2|   H|
|  7|dsgdb9nsd_000001|           3|           0|                 84.8093|   1|   C|
|  8|dsgdb9nsd_000001|           3|           4|                -11.2543|   

In [100]:
val targetCol = df2.columns.filter(_.startsWith("scalar")).head

targetCol: String = "scalar_coupling_constant"

#### Rename Columns using Map

In [102]:
df2 = Map(
    "atom_index_0" -> "a0", 
    "atom_index_1" -> "a1",
    "molecule_name" -> "name"
).foldLeft(df2)((d, m) => d.withColumnRenamed(m._1, m._2))

In [103]:
df2.show

+---+----------------+---+---+------------------------+----+----+
| id|            name| a0| a1|scalar_coupling_constant|t1_1|t1_4|
+---+----------------+---+---+------------------------+----+----+
|  0|dsgdb9nsd_000001|  1|  0|                 84.8076|   1|   C|
|  1|dsgdb9nsd_000001|  1|  2|                 -11.257|   2|   H|
|  2|dsgdb9nsd_000001|  1|  3|                -11.2548|   2|   H|
|  3|dsgdb9nsd_000001|  1|  4|                -11.2543|   2|   H|
|  4|dsgdb9nsd_000001|  2|  0|                 84.8074|   1|   C|
|  5|dsgdb9nsd_000001|  2|  3|                -11.2541|   2|   H|
|  6|dsgdb9nsd_000001|  2|  4|                -11.2548|   2|   H|
|  7|dsgdb9nsd_000001|  3|  0|                 84.8093|   1|   C|
|  8|dsgdb9nsd_000001|  3|  4|                -11.2543|   2|   H|
|  9|dsgdb9nsd_000001|  4|  0|                 84.8095|   1|   C|
| 10|dsgdb9nsd_000002|  1|  0|                 32.6889|   1|   N|
| 11|dsgdb9nsd_000002|  1|  2|                -11.1866|   2|   H|
| 12|dsgdb

In [108]:
df2 = df2.withColumn("adiff", abs(col("a0")-col("a1")))

In [111]:
var tempCols = Seq("a0", "a1", "adiff", "t1_1", "t1_4")

In [138]:
var tempAggs:Map[String, Column=>Column] = Map(
    "count" -> count,
    "min" -> max, 
    "max" -> min, 
    "mean"-> avg, 
    "stddev"-> stddev
)


tempAggs: Map[String, Column => Column] = Map(
  "count" -> <function1>,
  "mean" -> <function1>,
  "min" -> <function1>,
  "max" -> <function1>,
  "stddev" -> <function1>
)

In [139]:
var tempExpr = tempAggs.map(x => x._2(col(targetCol)) ).toSeq

tempExpr: Seq[Column] = List(
  count(scalar_coupling_constant),
  avg(scalar_coupling_constant),
  max(scalar_coupling_constant),
  min(scalar_coupling_constant),
  stddev_samp(scalar_coupling_constant)
)

In [167]:
import scala.util.matching.Regex.Match
def replacePars(str:String) = "[(|)]".r replaceSomeIn(str, { 
  case Match("(") => Some("_") 
  case Match(")") => Some("") 
  case _ => None})
def replaceColumnNameChars(df:DataFrame):DataFrame = {
   df.select(df.columns.map(x => col(x).as(replacePars(x))):_*)
}

import scala.util.matching.Regex.Match

defined function replacePars
defined function replaceColumnNameChars

In [173]:
val aggrList01 = tempCols.map(x => df2.groupBy(x).agg(tempExpr.head, tempExpr.tail:_*).
                                transform(replaceColumnNameChars).sort(col("avg_"+targetCol).desc))

aggrList01: Seq[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = List(
  [a0: string, count_scalar_coupling_constant: bigint ... 4 more fields],
  [a1: string, count_scalar_coupling_constant: bigint ... 4 more fields],
  [adiff: double, count_scalar_coupling_constant: bigint ... 4 more fields],
  [t1_1: string, count_scalar_coupling_constant: bigint ... 4 more fields],
  [t1_4: string, count_scalar_coupling_constant: bigint ... 4 more fields]
)

In [174]:
aggrList01(1).show

+---+------------------------------+----------------------------+----------------------------+----------------------------+------------------------------------+
| a1|count_scalar_coupling_constant|avg_scalar_coupling_constant|max_scalar_coupling_constant|min_scalar_coupling_constant|stddev_samp_scalar_coupling_constant|
+---+------------------------------+----------------------------+----------------------------+----------------------------+------------------------------------+
|  0|                        322761|           46.93884210838372|                     99.9974|                   -0.000164|                  42.978647681959394|
|  8|                        248731|          23.526509590782968|                     99.9898|                   -0.000113|                  43.381887989068176|
|  7|                        359545|          20.845812907291148|                     99.9997|                   -0.000105|                   39.56350150932114|
|  6|                        39278

#### StringIndexer for StringType Columns

In [ ]:
import org.apache.spark.ml.feature.

In [104]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator
val encoder = new OneHotEncoderEstimator().
    setInputCols(Array("t1_1", "t1_4")).
    setOutputCols(Array("t1", "t4"))

import org.apache.spark.ml.feature.OneHotEncoderEstimator

encoder: OneHotEncoderEstimator = oneHotEncoder_5eebcffc9246

In [105]:
val model = encoder.fit(df2)
val encoded = model.transform(df2)
encoded.show()